# 06: Extending xsnow

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/06_extending_xsnow.ipynb)

This notebook explores xsnow's extension system and shows you how to create custom analysis functions.

## What You'll Learn

- Understanding xsnow's architecture
- The extension system and how it works
- Creating custom computed variables
- Registering new methods
- Contributing to xsnow
- Best practices for extensions


### Learning objectives
- Inspect xsnow's dataset architecture to understand extension entry points.
- Create lightweight computed variables and wrap them as reusable extension methods.
- Register and organize extensions so teams can share capabilities.
- Prototype an advanced extension that adds new dimensions or metadata.

**Prerequisites**
- [ ] Comfortable with notebooks 01–05.
- [ ] Experience writing Python functions and classes.
- [ ] Familiarity with xarray/xsnow data structures.


## Installation (For Colab Users)
**Show.** Install xsnow and developer dependencies when working in a hosted runtime.


In [ ]:
# Run.
%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow


## Setup: Load Reference Dataset
**Show.** Load a sample dataset so extension experiments have real data.


In [ ]:
# Run.
import xsnow
import numpy as np

reference = xsnow.single_profile_timeseries()
print('✅ Dataset ready:', dict(reference.dims))


**Explain.** A known dataset lets you verify extension behavior before sharing it widely.


In [ ]:
# Check for understanding: reference dataset
assert reference is not None
assert 'time' in reference.dims


## Part 1: Understanding xsnow's Architecture
**Show.** Explore dataset structure to identify extension hooks.


In [ ]:
# Run.
print('Coordinates:', list(reference.coords))
print('Variables:', list(reference.data_vars)[:5])
print('Available methods:', [m for m in dir(reference) if m.startswith('to_')][:5])


**Explain.** Knowing the coordinates, variables, and built-in helpers keeps custom code aligned with xsnow conventions.


In [ ]:
# Check for understanding: architecture
assert hasattr(reference, 'coords')
assert len(reference.data_vars) > 0


## Part 2: Creating Custom Computed Variables
**Show.** Derive a 24-hour new snow estimate from cumulative `HS`.


In [ ]:
# Run.
new_snow = reference['HS'].diff(dim='time')
new_snow = new_snow.fillna(0).clip(min=0)
reference_with_new = reference.assign(new_snow_24h=new_snow)
print(reference_with_new['new_snow_24h'].isel(location=0, slope=0, realization=0).to_series().head())


**Explain.** Diffing `HS` exposes fresh accumulation while clipping negative values guards against melt artifacts.


In [ ]:
# Check for understanding: new snow variable
assert 'new_snow_24h' in reference_with_new.data_vars
assert (reference_with_new['new_snow_24h'] >= 0).all()


## Part 3: Creating Extension Methods
**Show.** Package the computation as a reusable method.


In [ ]:
# Run.
def compute_new_snow(ds, hours=24):
    # Return positive HS changes over the previous window
    rolling = ds['HS'].diff(dim='time')
    return rolling.fillna(0).clip(min=0)

class SnowExtensions:
    @staticmethod
    def new_snow(ds, hours=24):
        return compute_new_snow(ds, hours=hours)

preview = SnowExtensions.new_snow(reference).isel(location=0, slope=0, realization=0).to_series().head()
print(preview)


**Explain.** Wrapping logic in a helper class makes it easy to register with xsnow or import across projects.


In [ ]:
# Check for understanding: extension helper
assert callable(SnowExtensions.new_snow)
assert (preview >= 0).all()


## Part 4: Extension Registration
**Show.** Attach the helper to `xsnow.Dataset` so users can call it as a method.


In [ ]:
# Run.
from types import MethodType

def register_extension():
    def new_snow_method(self, hours=24):
        return SnowExtensions.new_snow(self, hours=hours)
    xsnow.dataset.Dataset.new_snow = MethodType(new_snow_method, xsnow.dataset.Dataset)

register_extension()
print('Registered new_snow:', hasattr(reference, 'new_snow'))


**Explain.** Registration converts your helper into a first-class dataset method with minimal boilerplate.


In [ ]:
# Check for understanding: registration
assert hasattr(xsnow.dataset.Dataset, 'new_snow')


## Part 5: Contributing to xsnow
**Show.** Generate a checklist to prepare contributions.


In [ ]:
# Run.
contribution_steps = [
    'Fork the repository and create a feature branch.',
    'Add your extension with docstrings and tests.',
    'Run the existing test suite and lint checks.',
    'Open a pull request referencing relevant issues.'
]
for step in contribution_steps:
    print('•', step)


**Explain.** Treat extensions like production code—tests, docs, and reviews keep quality high.


In [ ]:
# Check for understanding: contribution list
assert len(contribution_steps) == 4


## Part 6: Advanced Extension Example
**Show.** Prototype an extension that tags layers exceeding a density threshold.


In [ ]:
# Run.
def tag_dense_layers(ds, threshold=350):
    mask = ds['density'] > threshold
    return ds.assign_coords(dense_layer=mask)

advanced = tag_dense_layers(reference)
print('Dense layer coordinate added:', 'dense_layer' in advanced.coords)
print('Sample:', advanced['dense_layer'].isel(location=0, slope=0, realization=0).values[:5])


**Explain.** Adding coordinates or metadata opens the door to richer filtering and visualization workflows.


In [ ]:
# Check for understanding: advanced extension
assert 'dense_layer' in advanced.coords


### Play
Tweak thresholds or rolling windows to see how extensions respond.


In [ ]:
# Run.
new_snow_window = 12  # Try 6, 12, 24
threshold = 300  # Try 250–400

new_snow_custom = SnowExtensions.new_snow(reference, hours=new_snow_window)
dense_tag_custom = tag_dense_layers(reference, threshold=threshold)
print('Window:', new_snow_window, 'Threshold:', threshold)
print('Mean new snow:', float(new_snow_custom.mean()))
print('Dense layers flagged:', int(dense_tag_custom['dense_layer'].sum()))


## Practice
Apply what you learned before opening the solutions.


1. Write an extension that calculates the ratio of `density` to `temperature` for each layer.
2. Register a method that returns the maximum `HS` over a configurable time window.
3. Draft documentation text explaining when to use your new extensions.


<details>
<summary>Solutions</summary>

1. Create `density_temp_ratio = ds['density'] / ds['temperature']` and handle divide-by-zero.
2. Implement `def max_hs(self, window=3): return self['HS'].rolling(time=window).max()` and register like above.
3. Describe problem scenarios, inputs, and expected outputs in your README or docstrings.

</details>


## Summary
- Understanding xsnow's structure reveals natural extension hooks.
- Encapsulate computations as helpers, then register them for team-wide reuse.
- Iterating with play prompts and practice keeps extensions maintainable.
